<a href="https://colab.research.google.com/github/AfraHossain/Neural-network/blob/main/Lenet_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import datetime
import torchvision
import torch
from torch import nn, optim
from torchvision import datasets, transforms
import torch.nn.functional as F

In [ ]:
device = ("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_dataset_path="/content/drive/MyDrive/Dataset/train"
test_dataset_path="/content/drive/MyDrive/Dataset/test"

In [ ]:
transform = transforms.Compose([
          transforms.Resize((32, 32)),
          transforms.ToTensor()
          ])

train_set = torchvision.datasets.ImageFolder(root = train_dataset_path, transform = transform)
trainloader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)
test_set = torchvision.datasets.ImageFolder(root = test_dataset_path, transform = transform)
testloader = torch.utils.data.DataLoader(test_set, batch_size=64, shuffle=True)

train_data_size = len(train_set)
test_data_size = len(test_set)

In [ ]:
training_data = enumerate(trainloader)
batch_idx, (images, labels) = next(training_data)
print(images.shape) # Size of the image
print(labels.shape) # Size of the labels

torch.Size([64, 3, 32, 32])
torch.Size([64])


In [ ]:
class LeNet5(nn.Module):

    def __init__(self):
        super(LeNet5, self).__init__()
        
        self.convolutional_layer = nn.Sequential(            
            nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2, stride=2, padding=0),
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2, stride=2, padding=0),
            nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, stride=1),
            nn.Tanh()
        )

        self.linear_layer = nn.Sequential(
            nn.Linear(in_features=120, out_features=84),
            nn.Tanh(),
            nn.Linear(in_features=84, out_features=4),
        )


    def forward(self, x):
        x = self.convolutional_layer(x)
        x = torch.flatten(x, 1)
        x = self.linear_layer(x)
        x = F.softmax(x, dim=1)
        return x

In [ ]:
model = LeNet5().to(device)
print(model)

LeNet5(
  (convolutional_layer): Sequential(
    (0): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): Tanh()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): Tanh()
    (5): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (6): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
    (7): Tanh()
  )
  (linear_layer): Sequential(
    (0): Linear(in_features=120, out_features=84, bias=True)
    (1): Tanh()
    (2): Linear(in_features=84, out_features=4, bias=True)
  )
)


In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [ ]:
epochs = 30
train_loss, val_loss = [], []

for epoch in range(epochs):
   
    total_train_loss = 0
    total_val_loss = 0

    model.train()
    
    # training our model
    for idx, (image, label) in enumerate(trainloader):

        image, label = image.to(device), label.to(device)
        optimizer.zero_grad()
        pred = model(image)

        loss = criterion(pred, label)
        total_train_loss += loss.item()

        loss.backward()
        optimizer.step()

    total_train_loss = total_train_loss / (idx + 1)
    train_loss.append(total_train_loss)
    
    # validating our model
    model.eval()
    total = 0
    for idx, (image, label) in enumerate(testloader):
        image, label = image.to(device), label.to(device)
        pred = model(image)
        loss = criterion(pred, label)
        total_val_loss += loss.item()

        pred = torch.nn.functional.softmax(pred, dim=1)
        for i, p in enumerate(pred):
            if label[i] == torch.max(p.data, 0)[1]:
                total = total + 1

    accuracy = total / test_data_size

    total_val_loss = total_val_loss / (idx + 1)
    val_loss.append(total_val_loss)
    print(f"Epoch: {epoch} | Train Loss: {total_train_loss} | Val Loss: {total_val_loss} | Acc: {accuracy}")

Epoch: 0 | Train Loss: 1.1863351279554275 | Val Loss: 1.3556555358190385 | Acc: 0.355
Epoch: 1 | Train Loss: 1.079248150316671 | Val Loss: 1.3999658955468073 | Acc: 0.32475
Epoch: 2 | Train Loss: 1.0527846632293238 | Val Loss: 1.424151134869409 | Acc: 0.299
Epoch: 3 | Train Loss: 1.0355605475437908 | Val Loss: 1.3934074886261472 | Acc: 0.341
Epoch: 4 | Train Loss: 1.0235455698860338 | Val Loss: 1.4032640873439728 | Acc: 0.323
Epoch: 5 | Train Loss: 1.0060054659843445 | Val Loss: 1.4071075310782781 | Acc: 0.32775
Epoch: 6 | Train Loss: 0.9894096710430548 | Val Loss: 1.4108548523887756 | Acc: 0.3185
Epoch: 7 | Train Loss: 0.9768014003674443 | Val Loss: 1.429921585416037 | Acc: 0.2985
Epoch: 8 | Train Loss: 0.9597595183613201 | Val Loss: 1.3543312152226765 | Acc: 0.37525
Epoch: 9 | Train Loss: 0.9420512510945622 | Val Loss: 1.385264553720989 | Acc: 0.34175
Epoch: 10 | Train Loss: 0.9298336964826615 | Val Loss: 1.3349658194042386 | Acc: 0.39925
Epoch: 11 | Train Loss: 0.915520926824393 | V